In [ ]:
#pip install unidecode

In [2]:
#pip install --upgrade gensim

In [3]:
#pip install sklearn

In [1]:
#pip install translate

  Created wheel for distlib: filename=distlib-0.3.0-py3-none-any.whl size=340432 sha256=6df0e21ba075a18321ef5ffc5f67620418a6fd6d45c27169142a38c1e25744c7
  Stored in directory: c:\users\dell\appdata\local\pip\cache\wheels\a2\19\da\a15d4e2bedf3062c739b190d5cb5b7b2ecfbccb6b0d93c861b
Successfully built distlib
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
import re
import sklearn
from re import sub
import multiprocessing
from unidecode import unidecode

import gensim

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec
from gensim.test.utils import get_tmpfile
from gensim.models import KeyedVectors
from sklearn.cluster import KMeans

from time import time 
from collections import defaultdict

import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [2]:
file = pd.read_csv("USelectionw2v.csv")

In [3]:
file.count

<bound method DataFrame.count of                             created_at  \
0       Sat Jun 13 20:41:42 +0000 2020   
1       Sat Jun 13 20:41:42 +0000 2020   
2       Sat Jun 13 20:41:42 +0000 2020   
3       Sat Jun 13 20:41:42 +0000 2020   
4       Sat Jun 13 20:41:43 +0000 2020   
...                                ...   
102624  Sat Jun 13 16:07:41 +0000 2020   
102625  Sat Jun 13 16:07:41 +0000 2020   
102626  Sat Jun 13 16:07:41 +0000 2020   
102627  Sat Jun 13 16:07:41 +0000 2020   
102628  Sat Jun 13 16:07:41 +0000 2020   

                                                full_text  \
0             @traceybra @TheLondonHughes #AllLivesMatter   
1       RT @beardedfinance: You look like an EasyJet f...   
2       RT @larryelder: #alllivesmatter https://t.co/A...   
3       RT @EWindt: Lieve @NUnl  #AllLivesMatter #AllL...   
4       @realDonaldTrump Congrats to the graduates. Bu...   
...                                                   ...   
102624  So inspiring! Now  when doe

In [4]:
file_cleaned=file.drop(columns=['created_at','user_mentions'])

In [5]:
file_cleaned.reset_index(drop=True)

,full_text,hashtags
0,@traceybra @TheLondonHughes #AllLivesMatter,AllLivesMatter
1,RT @beardedfinance: You look like an EasyJet f...,NaN
2,RT @larryelder: #alllivesmatter https://t.co/A...,alllivesmatter
3,RT @EWindt: Lieve @NUnl #AllLivesMatter #AllL...,"AllLivesMatter,AllLivesMatter,AllLivesMatter,A..."
4,@realDonaldTrump Congrats to the graduates. Bu...,NaN
...,...,...
102624,So inspiring! Now when does the real #Preside...,"President,PresidentBiden,TrumpIsANationalDisgrace"
102625,RT @jaimetoons: A white couple call the police...,BLM
102626,RT @sirjamesa12: Every single letter was desig...,NaN
102627,RT @YourAnonCentral: Recognize and acknowledge...,"NoJusticeNoPeace,justiceforNaledi,femicide"


In [6]:
file_cleaned.count

<bound method DataFrame.count of                                                 full_text  \
0             @traceybra @TheLondonHughes #AllLivesMatter   
1       RT @beardedfinance: You look like an EasyJet f...   
2       RT @larryelder: #alllivesmatter https://t.co/A...   
3       RT @EWindt: Lieve @NUnl  #AllLivesMatter #AllL...   
4       @realDonaldTrump Congrats to the graduates. Bu...   
...                                                   ...   
102624  So inspiring! Now  when does the real #Preside...   
102625  RT @jaimetoons: A white couple call the police...   
102626  RT @sirjamesa12: Every single letter was desig...   
102627  RT @YourAnonCentral: Recognize and acknowledge...   
102628  RT @Number10cat: Given Johnson feels it’s acce...   

                                                 hashtags  
0                                          AllLivesMatter  
1                                                     NaN  
2                                          alllivesmat

In [7]:
file_cleaned = file_cleaned.drop_duplicates().reset_index(drop=True)

In [8]:
file_cleaned.count

<bound method DataFrame.count of                                                full_text  \
0            @traceybra @TheLondonHughes #AllLivesMatter   
1      RT @beardedfinance: You look like an EasyJet f...   
2      RT @larryelder: #alllivesmatter https://t.co/A...   
3      RT @EWindt: Lieve @NUnl  #AllLivesMatter #AllL...   
4      @realDonaldTrump Congrats to the graduates. Bu...   
...                                                  ...   
37232  RT @Liberal_Leigh: Just minor point here  but ...   
37233  RT @ArtieVandelay1: @jaimetoons Good work  Twi...   
37234  @MOH_Scandals #الكويت #الغاء_منصب_نواب_القبايل...   
37235  RT @BrotherhoodUK: We're not saying others don...   
37236  So inspiring! Now  when does the real #Preside...   

                                                hashtags  
0                                         AllLivesMatter  
1                                                    NaN  
2                                         alllivesmatter  
3      All

In [9]:
file_cleaned['hashtags']=file_cleaned['hashtags'].str.lower()

In [10]:
def text_to_word_list(text, process):
    ''' Pre process and convert texts to a list of words 
    method inspired by method from eliorc github repo: https://github.com/eliorc/Medium/blob/master/MaLSTM.ipynb'''
    text = process(text)
    text = str(text)
    text = text.lower()

    # Clean the text
    text = sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)(?:(?:\/[^\s/]*))*'," ",text)
    text = sub(r'\w +@[a-zA-Z]\S+ '," ",text)
    text = sub(r"[^A-Za-z0-9^,!?.\/'+]", " ", text)
    text = sub(r"\+", " plus ", text)
    text = sub(r",", " ", text)
    text = sub(r"\.", " ", text)
    text = sub(r"!", " ", text)
    text = sub(r"\?", " ", text)
    text = sub(r"'", " ", text)
    text = sub(r":", " : ", text)
    text = sub(r"\s{2,}", " ", text)
    
    text = text.split()

    return text

In [11]:
file_cleaned.full_text = file_cleaned.full_text.apply(lambda x: text_to_word_list(x, unidecode))

In [12]:
import nltk
nltk.download('stopwords')
import csv
import io
import string

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
from nltk.corpus import stopwords
stop_words=set(stopwords.words('english'))

In [14]:
import re
l1=[]
l2=[]
l3=['no','nor','couldn','hasn','hadn','haven','didn','but']
for i in stop_words:
    str=''
    x=re.findall('(.*n.t$)',i)
    if len(x)!=0:
        a=str.join(x)
        l2.append(a)

for i in stop_words:
    if not i in l2:
        if i not in l3:
            l1.append(i)

In [15]:
file_cleaned.full_text=file_cleaned.full_text.apply(lambda x: [item for item in x if item not in l1])

In [16]:
file_model = file_cleaned.copy()

In [17]:
file_model = file_model[file_model.full_text.str.len()>1]

In [18]:
sent = [row for row in file_model.full_text]
phrases = Phrases(sent, min_count=5, progress_per=10000)
bigram = Phraser(phrases)
sentences = bigram[sent]
sentences[1]

INFO - 20:31:15: collecting all words and their counts
INFO - 20:31:15: PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO - 20:31:16: PROGRESS: at sentence #10000, processed 98956 words and 82928 word types
INFO - 20:31:16: PROGRESS: at sentence #20000, processed 189074 words and 143890 word types
INFO - 20:31:16: PROGRESS: at sentence #30000, processed 278097 words and 203511 word types
INFO - 20:31:17: collected 243088 word types from a corpus of 334727 words (unigram + bigrams) and 35693 sentences
INFO - 20:31:17: using 243088 counts as vocab in Phrases<0 vocab, min_count=5, threshold=10.0, max_vocab_size=40000000>
INFO - 20:31:17: source_vocab length 243088
INFO - 20:31:21: Phraser built with 2128 phrasegrams


['r', 'look_like', 'easyjet_flight', 'attendant']

In [19]:
w2v_model = Word2Vec(min_count=10,
                     window=4,
                     size=300,
                     sample=1e-5, 
                     alpha=0.07, 
                     min_alpha=0.0007, 
                     negative=4,
                     workers=multiprocessing.cpu_count()-1)

start = time()

w2v_model.build_vocab(sentences, progress_per=30000)

print('Time to build vocab: {} mins'.format(round((time() - start) / 50, 2)))

INFO - 20:31:54: collecting all words and their counts
INFO - 20:31:54: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 20:31:56: PROGRESS: at sentence #30000, processed 256801 words, keeping 42358 word types
INFO - 20:31:56: collected 48550 word types from a corpus of 309592 raw words and 35693 sentences
INFO - 20:31:56: Loading a fresh vocabulary
INFO - 20:31:56: effective_min_count=10 retains 4729 unique words (9% of original 48550, drops 43821)
INFO - 20:31:56: effective_min_count=10 leaves 222096 word corpus (71% of original 309592, drops 87496)
INFO - 20:31:56: deleting the raw counts dictionary of 48550 items
INFO - 20:31:56: sample=1e-05 downsamples 4729 most-common words
INFO - 20:31:56: downsampling leaves estimated 49731 word corpus (22.4% of prior 222096)
INFO - 20:31:56: estimated required memory for 4729 words and 300 dimensions: 13714100 bytes
INFO - 20:31:56: resetting layer weights


Time to build vocab: 0.09 mins


In [20]:
start = time()

w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=60, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - start) / 60, 2)))

w2v_model.init_sims(replace=True)

INFO - 20:32:02: training model with 7 workers on 4729 vocabulary and 300 features, using sg=0 hs=0 sample=1e-05 negative=4 window=4
INFO - 20:32:03: EPOCH 1 - PROGRESS: at 30.74% examples, 15554 words/s, in_qsize 0, out_qsize 0
INFO - 20:32:04: EPOCH 1 - PROGRESS: at 58.12% examples, 14227 words/s, in_qsize 0, out_qsize 0
INFO - 20:32:05: EPOCH 1 - PROGRESS: at 90.85% examples, 14745 words/s, in_qsize 0, out_qsize 0
INFO - 20:32:05: worker thread finished; awaiting finish of 6 more threads
INFO - 20:32:05: worker thread finished; awaiting finish of 5 more threads
INFO - 20:32:05: worker thread finished; awaiting finish of 4 more threads
INFO - 20:32:05: worker thread finished; awaiting finish of 3 more threads
INFO - 20:32:05: worker thread finished; awaiting finish of 2 more threads
INFO - 20:32:05: worker thread finished; awaiting finish of 1 more threads
INFO - 20:32:05: worker thread finished; awaiting finish of 0 more threads
INFO - 20:32:05: EPOCH - 1 : training on 309592 raw wo

INFO - 20:32:31: worker thread finished; awaiting finish of 3 more threads
INFO - 20:32:31: worker thread finished; awaiting finish of 2 more threads
INFO - 20:32:31: worker thread finished; awaiting finish of 1 more threads
INFO - 20:32:31: worker thread finished; awaiting finish of 0 more threads
INFO - 20:32:31: EPOCH - 10 : training on 309592 raw words (49396 effective words) took 2.8s, 17828 effective words/s
INFO - 20:32:32: EPOCH 11 - PROGRESS: at 33.86% examples, 16937 words/s, in_qsize 0, out_qsize 0
INFO - 20:32:33: EPOCH 11 - PROGRESS: at 74.87% examples, 17713 words/s, in_qsize 0, out_qsize 0
INFO - 20:32:33: worker thread finished; awaiting finish of 6 more threads
INFO - 20:32:33: worker thread finished; awaiting finish of 5 more threads
INFO - 20:32:33: worker thread finished; awaiting finish of 4 more threads
INFO - 20:32:33: worker thread finished; awaiting finish of 3 more threads
INFO - 20:32:33: worker thread finished; awaiting finish of 2 more threads
INFO - 20:32:

INFO - 20:32:59: EPOCH 20 - PROGRESS: at 74.87% examples, 17713 words/s, in_qsize 0, out_qsize 0
INFO - 20:33:00: worker thread finished; awaiting finish of 6 more threads
INFO - 20:33:00: worker thread finished; awaiting finish of 5 more threads
INFO - 20:33:00: worker thread finished; awaiting finish of 4 more threads
INFO - 20:33:00: worker thread finished; awaiting finish of 3 more threads
INFO - 20:33:00: worker thread finished; awaiting finish of 2 more threads
INFO - 20:33:00: worker thread finished; awaiting finish of 1 more threads
INFO - 20:33:00: worker thread finished; awaiting finish of 0 more threads
INFO - 20:33:00: EPOCH - 20 : training on 309592 raw words (49684 effective words) took 2.7s, 18422 effective words/s
INFO - 20:33:01: EPOCH 21 - PROGRESS: at 36.97% examples, 17618 words/s, in_qsize 0, out_qsize 0
INFO - 20:33:02: EPOCH 21 - PROGRESS: at 78.41% examples, 17944 words/s, in_qsize 0, out_qsize 0
INFO - 20:33:03: worker thread finished; awaiting finish of 6 more

INFO - 20:33:26: EPOCH - 29 : training on 309592 raw words (49709 effective words) took 3.1s, 16081 effective words/s
INFO - 20:33:27: EPOCH 30 - PROGRESS: at 30.74% examples, 15255 words/s, in_qsize 0, out_qsize 0
INFO - 20:33:28: EPOCH 30 - PROGRESS: at 71.46% examples, 16770 words/s, in_qsize 0, out_qsize 0
INFO - 20:33:29: worker thread finished; awaiting finish of 6 more threads
INFO - 20:33:29: worker thread finished; awaiting finish of 5 more threads
INFO - 20:33:29: worker thread finished; awaiting finish of 4 more threads
INFO - 20:33:29: worker thread finished; awaiting finish of 3 more threads
INFO - 20:33:29: worker thread finished; awaiting finish of 2 more threads
INFO - 20:33:29: worker thread finished; awaiting finish of 1 more threads
INFO - 20:33:29: worker thread finished; awaiting finish of 0 more threads
INFO - 20:33:29: EPOCH - 30 : training on 309592 raw words (50036 effective words) took 2.8s, 17566 effective words/s
INFO - 20:33:30: EPOCH 31 - PROGRESS: at 36.9

INFO - 20:33:55: worker thread finished; awaiting finish of 2 more threads
INFO - 20:33:55: worker thread finished; awaiting finish of 1 more threads
INFO - 20:33:55: worker thread finished; awaiting finish of 0 more threads
INFO - 20:33:55: EPOCH - 39 : training on 309592 raw words (49848 effective words) took 2.9s, 17146 effective words/s
INFO - 20:33:56: EPOCH 40 - PROGRESS: at 24.63% examples, 11443 words/s, in_qsize 0, out_qsize 0
INFO - 20:33:57: EPOCH 40 - PROGRESS: at 61.66% examples, 14197 words/s, in_qsize 0, out_qsize 0
INFO - 20:33:58: worker thread finished; awaiting finish of 6 more threads
INFO - 20:33:58: worker thread finished; awaiting finish of 5 more threads
INFO - 20:33:58: worker thread finished; awaiting finish of 4 more threads
INFO - 20:33:58: worker thread finished; awaiting finish of 3 more threads
INFO - 20:33:58: worker thread finished; awaiting finish of 2 more threads
INFO - 20:33:58: worker thread finished; awaiting finish of 1 more threads
INFO - 20:33:

INFO - 20:34:24: worker thread finished; awaiting finish of 6 more threads
INFO - 20:34:24: worker thread finished; awaiting finish of 5 more threads
INFO - 20:34:24: worker thread finished; awaiting finish of 4 more threads
INFO - 20:34:24: worker thread finished; awaiting finish of 3 more threads
INFO - 20:34:24: worker thread finished; awaiting finish of 2 more threads
INFO - 20:34:24: worker thread finished; awaiting finish of 1 more threads
INFO - 20:34:24: worker thread finished; awaiting finish of 0 more threads
INFO - 20:34:24: EPOCH - 49 : training on 309592 raw words (49482 effective words) took 2.9s, 17150 effective words/s
INFO - 20:34:25: EPOCH 50 - PROGRESS: at 30.74% examples, 14548 words/s, in_qsize 0, out_qsize 0
INFO - 20:34:26: EPOCH 50 - PROGRESS: at 71.46% examples, 16393 words/s, in_qsize 0, out_qsize 0
INFO - 20:34:27: worker thread finished; awaiting finish of 6 more threads
INFO - 20:34:27: worker thread finished; awaiting finish of 5 more threads
INFO - 20:34:

INFO - 20:34:51: EPOCH - 58 : training on 309592 raw words (49687 effective words) took 2.9s, 17357 effective words/s
INFO - 20:34:52: EPOCH 59 - PROGRESS: at 36.97% examples, 17919 words/s, in_qsize 0, out_qsize 0
INFO - 20:34:53: EPOCH 59 - PROGRESS: at 78.41% examples, 18708 words/s, in_qsize 0, out_qsize 0
INFO - 20:34:53: worker thread finished; awaiting finish of 6 more threads
INFO - 20:34:53: worker thread finished; awaiting finish of 5 more threads
INFO - 20:34:53: worker thread finished; awaiting finish of 4 more threads
INFO - 20:34:53: worker thread finished; awaiting finish of 3 more threads
INFO - 20:34:53: worker thread finished; awaiting finish of 2 more threads
INFO - 20:34:53: worker thread finished; awaiting finish of 1 more threads
INFO - 20:34:53: worker thread finished; awaiting finish of 0 more threads
INFO - 20:34:53: EPOCH - 59 : training on 309592 raw words (50243 effective words) took 2.6s, 19011 effective words/s
INFO - 20:34:54: EPOCH 60 - PROGRESS: at 33.8

Time to train the model: 2.91 mins


In [21]:
w2v_model.save("word2vec.model")

INFO - 20:34:58: saving Word2Vec object under word2vec.model, separately None
INFO - 20:34:58: not storing attribute vectors_norm
INFO - 20:34:58: not storing attribute cum_table
INFO - 20:34:59: saved word2vec.model


In [22]:
pairs = [
    ('trump', 'donaldtrump'), 
    ('trump', 'donaldtrump'),
    ('joebiden', 'biden'),
]
for w1, w2 in pairs:
    print('%r\t%r\t%.2f' % (w1, w2, w2v_model.wv.similarity(w1, w2)))

'trump'	'donaldtrump'	0.62
'trump'	'donaldtrump'	0.62
'joebiden'	'biden'	0.85


In [23]:
w2v_model.wv.most_similar('election2020')

[('2020election', 0.9433926939964294),
 ('rememberinnovember', 0.9417142868041992),
 ('vote_voteouthate', 0.9357298612594604),
 ('electionday', 0.9241092801094055),
 ('soon_enough', 0.9234402179718018),
 ('winning', 0.9220186471939087),
 ('november_3rd', 0.9188413619995117),
 ('votehimout', 0.9181077480316162),
 ('voteblue', 0.9179947376251221),
 ('best_yet', 0.9107718467712402)]

In [24]:
file_export = file_model.copy()
file_export['old_full_text'] = file_export.full_text
file_export.old_full_text = file_export.old_full_text.str.join(' ')
file_export.full_text = file_export.full_text.apply(lambda x: ' '.join(bigram[x]))

In [25]:
file_export[['full_text','hashtags']].to_csv('cleaned_dataset.csv', index=False)

In [26]:
word_vectors = Word2Vec.load("word2vec.model").wv

INFO - 20:35:11: loading Word2Vec object from word2vec.model
INFO - 20:35:12: loading wv recursively from word2vec.model.wv.* with mmap=None
INFO - 20:35:12: setting ignored attribute vectors_norm to None
INFO - 20:35:12: loading vocabulary recursively from word2vec.model.vocabulary.* with mmap=None
INFO - 20:35:12: loading trainables recursively from word2vec.model.trainables.* with mmap=None
INFO - 20:35:12: setting ignored attribute cum_table to None
INFO - 20:35:12: loaded word2vec.model
